In [7]:
import pandas as pd
import polars as pl

---

In [4]:
meta_data = pd.read_parquet("/Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/5af0c3ea-c280-59e5-83f0-bcf41247b3ab/index_files/train_index_meta.parquet")
meta_data

,file_path,count,start_row_index
0,/Users/cooper/Desktop/hydro-forecasting/tests/...,4020,0
1,/Users/cooper/Desktop/hydro-forecasting/tests/...,1187,4020
2,/Users/cooper/Desktop/hydro-forecasting/tests/...,4090,5207
3,/Users/cooper/Desktop/hydro-forecasting/tests/...,3547,9297
4,/Users/cooper/Desktop/hydro-forecasting/tests/...,4090,12844
5,/Users/cooper/Desktop/hydro-forecasting/tests/...,3599,16934
6,/Users/cooper/Desktop/hydro-forecasting/tests/...,4167,20533
7,/Users/cooper/Desktop/hydro-forecasting/tests/...,4167,24700
8,/Users/cooper/Desktop/hydro-forecasting/tests/...,3932,28867
9,/Users/cooper/Desktop/hydro-forecasting/tests/...,6313,32799


In [6]:
data = pd.read_parquet("/Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/5af0c3ea-c280-59e5-83f0-bcf41247b3ab/index_files/train_index.parquet")
data

,file_path,gauge_id,start_idx,end_idx,input_end_date,valid_sequence,stage,static_file_path
0,/Users/cooper/Desktop/hydro-forecasting/tests/...,CA_15013,0,44,9.496224e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
1,/Users/cooper/Desktop/hydro-forecasting/tests/...,CA_15013,1,45,9.497088e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
2,/Users/cooper/Desktop/hydro-forecasting/tests/...,CA_15013,2,46,9.497952e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
3,/Users/cooper/Desktop/hydro-forecasting/tests/...,CA_15013,3,47,9.498816e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
4,/Users/cooper/Desktop/hydro-forecasting/tests/...,CA_15013,4,48,9.499680e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
...,...,...,...,...,...,...,...,...
94877,/Users/cooper/Desktop/hydro-forecasting/tests/...,USA_01073000,6312,6356,8.637408e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
94878,/Users/cooper/Desktop/hydro-forecasting/tests/...,USA_01073000,6313,6357,8.638272e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
94879,/Users/cooper/Desktop/hydro-forecasting/tests/...,USA_01073000,6314,6358,8.639136e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...
94880,/Users/cooper/Desktop/hydro-forecasting/tests/...,USA_01073000,6315,6359,8.640000e+17,True,train,/Users/cooper/Desktop/hydro-forecasting/tests/...


In [9]:
df = pl.scan_parquet("/Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/5af0c3ea-c280-59e5-83f0-bcf41247b3ab/index_files/train_index_meta.parquet")


result = (
    df.tail(1)  # take the last row
    .select(
        (pl.col("count") + pl.col("start_row_index")).alias("sum_count_start_row_index")
    )
    .collect()
)

print(result)

shape: (1, 1)
┌───────────────────────────┐
│ sum_count_start_row_index │
│ ---                       │
│ u32                       │
╞═══════════════════════════╡
│ 94882                     │
└───────────────────────────┘


In [11]:
import duckdb

# 1. Connect (in‐memory by default)
con = duckdb.connect()

# 2. Point at your Parquet file
path = "/Users/cooper/Desktop/hydro-forecasting/tests/yolo_6/5af0c3ea-c280-59e5-83f0-bcf41247b3ab/index_files/train_index_meta.parquet"

# 3. SQL to compute your sum on the very last row
query = f"""
WITH meta AS (
  SELECT * FROM read_parquet('{path}')
)
SELECT
  (count + start_row_index) AS sum_count_start_row_index
FROM meta
LIMIT 1
OFFSET (SELECT COUNT(*) - 1 FROM meta);
"""

# 4. Execute and grab the single value
# Option A: using fetchone()[0]
value = con.execute(query).fetchone()[0]

# Option B: if you’re on DuckDB ≥0.8.1, you can do:
# value = con.execute(query).fetch_scalar()

print(value)


94882
